# Install and Import Dependencies

In [34]:
%pip install opencv-python
%pip install mediapipe
%pip install tensorflow
%pip install matplotlib
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: opencv-python in c:\users\lance aziz\appdata\local\programs\python\python311\lib\site-packages (4.9.0.80)




[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [35]:
import cv2
import mediapipe as mp
import numpy as np
import os
from matplotlib import pyplot as plt

# Extracting and Drawing Joints
<img src="https://www.researchgate.net/profile/Jungpil-Shin/publication/362351225/figure/fig3/AS:1183521695645696@1659184969422/Mediapipe-detects-33-nodes-of-the-human-pose.ppm" width="500px">

In [36]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [37]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [38]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections

In [39]:
cap = cv2.VideoCapture(1)
with  mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened(): 
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        
        # Render detections
        draw_landmarks(image, results)

        cv2.imshow('Human GAIT Identification', image)
        
        if cv2.waitKey(10) & 0xFF == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

In [40]:
#number of detected landmarks
len(results.pose_landmarks.landmark)

33

In [41]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose

In [42]:
result_test = extract_keypoints(results)
result_test

array([ 0.47889575,  0.42884722, -1.85827088,  0.99393922,  0.51603395,
        0.32171446, -1.80196881,  0.99635154,  0.54434234,  0.31602228,
       -1.80166852,  0.99676853,  0.56813961,  0.31252432, -1.80120969,
        0.99757528,  0.4120234 ,  0.34933576, -1.81931937,  0.99373204,
        0.38384658,  0.3597838 , -1.81885886,  0.99230891,  0.36217746,
        0.3699007 , -1.81890643,  0.99217397,  0.61867481,  0.34010425,
       -1.34041798,  0.99785268,  0.32227725,  0.42545608, -1.39168835,
        0.99166852,  0.55078208,  0.51425922, -1.67138612,  0.98741502,
        0.44059649,  0.54536861, -1.68845534,  0.9736352 ,  0.81875068,
        0.6942746 , -0.93111718,  0.9956044 ,  0.23396042,  0.88206303,
       -1.09565103,  0.97860867,  0.89278185,  0.98938727, -1.04918158,
        0.395311  ,  0.16539007,  1.39824593, -1.14281404,  0.45534343,
        0.95430738,  1.43002701, -1.73533702,  0.13377149,  0.21529381,
        1.75264227, -1.74237227,  0.14876814,  0.98701686,  1.55

# 4. Setup Folders for Collection

In [43]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Temp_Dataset') 

# Persons that we try to detect
persons = np.array(['Lance'])

#Very person with 4 different view
views = np.array(['Front','Back','Left','Right'])

# 10 videos worth of data
no_sequences = 20

# Videos are going to be 30 frames in length
sequence_length = 60

In [44]:
for person in persons:
    for view in views:
        for sequence in range(no_sequences):
            try: 
                os.makedirs(os.path.join(DATA_PATH, person, view, str(sequence)))
            except:
                pass

# Capturing Dataset

In [45]:
cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through persons
    for person in persons:
        ## Loop through views
        for view in views:
           for sequence in range(no_sequences):
                for frame_num in range(sequence_length):

                    # Read feed
                    ret, frame = cap.read()

                    # Make detections
                    image, results = mediapipe_detection(frame, holistic)

                    # Draw landmarks
                    draw_landmarks(image, results)
                    
                    # NEW Apply wait logic
                    if frame_num == 0: 
                        cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 1, cv2.LINE_AA)
                        cv2.putText(image, 'Collecting frames for {} Video for {} view frame Number #{}#'.format(person, view, sequence), (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        cv2.waitKey(3000)
                    else: 
                        cv2.putText(image, 'Collecting frames for {} Video for {} view frame Number #{}#'.format(person, view, sequence), (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        
                    # NEW Export keypoints
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, person, view, str(sequence), str(frame_num))
                    np.save(npy_path, keypoints)

                    
                    cv2.imshow('OpenCV Feed', image)

                    # Break gracefully
                    if cv2.waitKey(10) & 0xFF == 27:
                        break
                        
    cap.release()
    cv2.destroyAllWindows()

In [46]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [47]:
label_map = {label:num for num, label in enumerate(persons)}

In [48]:
label_map

{'Lance': 0}

In [49]:
sequences, labels = [], []
for person in persons:
    for view in views:
        for sequence in np.array(os.listdir(os.path.join(DATA_PATH, person, view))).astype(int):
            window = []
            for frame_num in range(sequence_length):

                res = np.load(os.path.join(DATA_PATH, person, view, str(sequence), "{}.npy".format(frame_num)))
                window.append(res)
            sequences.append(window)
            labels.append(label_map[person])

In [50]:
np.array(sequences).shape

(80, 60, 132)